In [1]:
# !pip3 install -U torch_geometric
# !pip3 install -U torch-scatter
# !pip3 install -U torch-sparse
# !pip3 install -U pandas
# !pip3 install -U numpy
# !pip3 install -U matplotlib
# !pip3 install -U sklearn
# !pip3 install -U PyTDC 
# !pip3 install -U pre-commit
# !pip3 install -U plotly
# !pip3 install -U tdc 
# !pip3 install -U ogb

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdkit

from ogb.graphproppred import PygGraphPropPredDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from tdc.single_pred import Tox
from rdkit.Chem import AllChem
from rdkit import Chem

from time import time

In [3]:
from featurizers.fingerprints import (
    MorganFingerprint,
    MACCSKeysFingerprint,
    AtomPairFingerprint,
    TopologicalTorsionFingerprint,
    ERGFingerprint,
)

In [20]:
dataset = PygGraphPropPredDataset(name = "ogbg-molhiv", root="./datasets")

ogbg-molhiv has been updated.
Will you update the dataset now? (y/N)
y


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [7]:
hiv_df = pd.read_csv(".\dataset\ogbg_molhiv\mapping\mol.csv.gz")

FileNotFoundError: [Errno 2] No such file or directory: '.\\dataset\\ogbg_molhiv\\mapping\\mol.csv.gz'

In [ ]:
hiv_df.head()

In [ ]:
X0 = hiv_df["smiles"]
y = hiv_df["HIV_active"]

In [ ]:
X1 = X0.apply(Chem.MolFromSmiles)

In [ ]:
X1_1 = [AllChem.GetMorganFingerprintAsBitVect(x, 2) for x in X1]
X1_1np = np.array(X1_1)
X1_1np

In [ ]:
X1_2 = X1.apply(AllChem.GetMorganFingerprintAsBitVect,args=(2,))
X1_2np = np.array(list(X1_2)) # // for some reason can't just accept X1_2 ...
X1_2np

In [ ]:
X = X1_2np

In [ ]:
y.value_counts().plot.bar()

Might be worth to use proper stratification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
clf = RandomForestClassifier()

start = time()
clf.fit(X_train, y_train)
end = time()

In [ ]:
print(f"training time: {end-start}s")

In [ ]:
print(f"AUROC: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}")